In [2]:
!pip install transformers einops accelerate bitsandbytes

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
  Using cached transformers-4.34.0-py3-none-any.whl.metadata (121 kB)
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/1e/2c/af22cd797fc368a9f098ed03015730e6568b884fe67f9940793d944a4b7b/bitsandbytes-0.41.1-py3-

In [3]:
!python -m pip install torch


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

In [6]:
import torch
import base64

In [19]:
checkpoint = "meta-llama/Llama-2-7b-chat-hf"

In [8]:
!pip install safetensors


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [9]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float32)

In [22]:
pipe = pipeline(
    'text2text-generation',
    model = base_model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16, 
    device_map ="auto",
    max_length = 256,
    do_sample = True,
    temperature = 0.3,
    top_p = 0.95
)
    

In [14]:
input_prompt = "give me a movie idea with depth and details"

In [15]:
model = pipe
generated_text = model(input_prompt)
generated_text

[{'generated_text': '"The Hunt" is a suspenseful thriller that follows a group of friends who are on a mission to find a lost treasure. Along the way, they encounter various obstacles and challenges, including a mysterious thief, a dangerous criminal organization, and a group of ruthless criminals who are determined to find the treasure. As they delve deeper into the mystery, they discover that the treasure is not just a mere treasure, but a powerful force that has been forged over centuries. The movie is a must-see for anyone who loves a good'}]

In [23]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	'HF_TASK':'text2text-generation',
    'device_map':'auto',
    'torch_dtype':'torch.float32'
}



huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, 
)

predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )

predictor.predict({
	"inputs": "how can I become more healthy?",
})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
-----------!

[{'generated_text': 'You can become more healthy by: 1. Eating a balanced and nutritious diet 2. Regular exercise 3.'}]

In [31]:
prompt = "write me a sci fi story with depth and details"

payload = {
    "inputs":prompt,
    "parameters": {
        "do_sample":True,
        "top_p":0.7,
        "temperature":0.3,
        "top_k":50,
        "max_new_tokens":512,
        "repetition_penalty":1.03
    }
}

response = predictor.predict(payload)

print(response)

[{'generated_text': 'In a distant galaxy, there lived a group of astronauts who were sent to explore a new planet. They were fascinated by the technology and the potential for advancements in science and technology. As they landed on the planet, they were amazed by the vastness of space and the vastness of the universe. They were amazed by the advanced technology and the incredible wonders it held. One day, they stumbled upon a mysterious object that seemed to be a portal to another world. As they explored the planet, they discovered that it was a portal to a different dimension. The object was so'}]


In [24]:
ENDPOINT = "huggingface-pytorch-tgi-inference-2023-10-18-17-24-49-299"

In [25]:
import boto3

In [27]:
runtime = boto3.client('runtime.sagemaker')


In [32]:
response = runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType="application/json", Body=json.dumps(payload))

In [33]:
print(response)

{'ResponseMetadata': {'RequestId': '0723dda4-865f-4ed9-9330-c9e81fd48777', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0723dda4-865f-4ed9-9330-c9e81fd48777', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 18 Oct 2023 21:14:06 GMT', 'content-type': 'application/json', 'content-length': '644', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f746505aec0>}


In [34]:
prediction = json.loads(response['Body'].read().decode('UTF-8'))